<a href="https://colab.research.google.com/github/mhtasnia/panda_practice/blob/main/Exercise_Creating%2C_Reading_and_Writing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'pitchfork-data:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F655%2F1252%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T074227Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D210e989490409a14dd48704e27c6b0978247a782620a67a0c3ecf481057d8a259446dd58093be9e4c64f9e0acd18063d16200f90a3241ffc5a8501e146e9d545c3cd38f8827e21a8f08365a4b74bdab31e438f80b701a4962cbeb514b3e031eada8a39001344dc74a4fa3e39ef8086430bc8b4d183163c4d84167b030f17dc3e7d3170ed92379c404e2fdb63b7205bb75ddc126abcc89a69db2c5bd09f46bbc9110e0d1db10f9fc85b41cf223fd5afb9e70c259df9aaa45d7401948a0680a6f8120f61665f20ca308b2428e4f24ae6bd8c0c564bb973314989e06a4ca37d71d153760665a5cc97de2ca625892759c405fda1a7929fe564c3a919d6722ea4db63,chess:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2321%2F3919%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T074227Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da3ffde5a5acb54f389ea5326922c7e9e64deb533ad07b2368a445db17953a3a305433fb73852a13de3becdd5f98382751a98779c5053c0b8713e7cfaa1077cca4fb7b5d75f36245060b3a8804f092b52898845e0ffec5c1b9aecb75d6bb11dbdfa8773b7402da9316e588e0642ecdb96d5314dcc8b29905817cb8cc457ea1219e6b84af806bde37090467583a0afd60ac79aeaaf868cefe2d208eab8b3c3361d6cb438c6b1178034d3a1b82ea6d200b934661134a674766cb166f07ca603cc581afea3190f383b107d9f3b8f1ea2ef365bfcf18a5ef736f6cba7cd86c79cfe7e41592885dbdad97f92d9115ebd8db81ba547edf28b512033140d5cdc5913f616,kepler-exoplanet-search-results:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2894%2F4877%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T074227Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Da85721340d7440a048ebe88979daba21252e6ca75963f0a3314c6e5dfe2c97caf307b035db8cf7c24a61c25d7213700a9dd56033cd424514f083580d989160a703f17419d9d7bd63748f5a0212d114e4b92b01e9a1ea2043b2fa9634cdc67d8f05ec4bab27eee89b70023e3f8e5ca3cf44656576d47bc70605ef8e8b227c725cfacc132728c90a1d363b266ec9ceb926d3512e4aa3998e77bd478ea784213c667cfc0df728a3e5d28f547c6618feb99eb37c59a8692a4c6848447e3031ac9b738fc9d4f49830401b013c564d4b48a20d2e87700bb0673291700a12c26e9fe7c75791120b1e32768733a1f733d8993c60a037c19e7f6507f6e88079377862dc2a,things-on-reddit:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F3491%2F5624%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T074227Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D60a3c2dfab1fbf9c0b893e55eb99d105c6bb77b6bf0aad4c52e989d1308b6d5f72f7d209f3a7c3ac36dbbdc4cba92e9dc5f9ff2c012e11385eed1552099808c017fbd268417ecc691282283be160ab1b44f61b5142f2a08a71f32856a00cfd25037cd4a322330408acc8fcde8bf7ddb471a07cbcabda8b90f769aa3955e783f82a6c77a194a5982cb3bd84a154eaa6b40d562d134540d77bd18164bef6e951ecb8c1b4da83c7dba6c745ee1d417c04fa031310ff7f0bc9d3ad985721399f4b8639256f5da7699e57521f9eace3e81374b6d37f85d984d4ff47fd5a129788a6202d3c9a5c2f6a400c78fc3c5de1f365abf454daf639be5d903796a9b1c92c6ab8,wine-reviews:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1442%2F8172%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T074227Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D37c95786c9931f260b076feb65d3d88dc014464adefe57972ec8da11aeda7ec78f002cbdc4c702b89687253d3392c327116b94f6f4da5815ba4093c31091c6db3714bdd6e4fb3d8f676fdd65e4645ef8cdc4600a2dd4e83b4d4a1537c8a711479b65acaedd3c9f6d6d19e0bdbcb67a17128c810cf58b35fdbad3308b3c1f00d01dad6405919dee7df40ebdc6fc943d317509f5ff226850e2530bc5bb4b78675958644ec43aea2b6b20764ec59c6075e70708e215c358d81f65b61815189832d4404a153c77f0158e42e9f38315b5887114665d3305a1d5a83a962ef106ca6f89100c3d1b4e9639d8ce61223e841b3b48661792a8dd5b6e8b5f03091822d2a009,ramen-ratings:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F9366%2F13206%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T074227Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D719fd37c77c04b89f2abff038a489e6928c3e519ec92b5b7d697c13ae7f641097e71feeace4dbb28382bc7706651b78616c33a811d4472f851a823a26145f8669be26e48171dcff9461e8d97019ad4b45318fcbd444f0c793da3e20745a8ccaab1c0288edde0a65d124899a18e147a4af3794b8883da33f9101647deea712616a560eef5792fd4feae17f25e1a02f5882bb0bbd01c3d984d7da909820ece3da4ddea0388598a4de502edad7f7f027e5e96522d6e19244ebb3a4feda14b05d32e86dee5e60160d13bd97b85c7069f77a660ac8a1213a6f0f809a72ad60ad0a49a2c557aa5fa1a843d35e38914b096fc43d009516b10d8b6adfcc66a0d19a26669,powerlifting-database:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F179555%2F403916%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T074227Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D3674f73c2f2b61191db0cd46b0e0822161f63e17d9563b02c608535fa4d411c41a8e6cc81ebdf1ca00c394d1fa40e7bb6be0803980d63dff624a2b7ad5fa27b20ed2ae21d07dbc8818b34fc7f1e65a6bbc87efe38accf48492498e313c2e5b1b039732d9c849bb042f13bdc6a7dd4f40fef7cd459dc75ac32889271ccf8b8baa31e829485b3560036da25b601c4578ae7becbf97514c020fb9181e9e1fef421b369a4b9307605b27938ec4ccf4506b17c33e7b59526af9912835a346b90033e134608ae719dd695dcda2cfac7454f181a96b8978e5f0617ab8ffeffcb298dc9bf05f73c4cf5f7c005094fee4adc8905dd9bd2e7cdd33ac91f1e83fa45e30e9a6,youtube-new:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F4549%2F466349%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T074227Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D94ccdd33835b16acd76bb65e9055c48a1f883b63617a1f565316f0a0457896cbb10a50971aeb115b4c7a21f291fe83067d1c4b3b5b5dca3aaa54e575457c9ccaf855b2c75061ca79bfd02cdf1f58ce90f0c80397526b1ef60b9ce0f7c0048ead4b0ed739b94ec5f7713ff9e2cc37e7bfeb48e4a3e730bc3f711a5b2bff028f0827b2ebb5a2a48e7911548ece9571f86bc35f4b9a6a913ece0f7af32cfc53559da4c63f851f7245993997536c7ed67c12258d1f0d1e881a024973542ee0d6c63a0e7bb5c46488e0f2038d83bc9eff0786c71a1324b0d3c2b63a8e452fc8ff63cacc3986fa193baf71991997c440f0dc65d1416758754e0fdf878262e35fa5db8e,188-million-us-wildfires:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F2478%2F1151655%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T074227Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2d107cff3248e6a50e6104d940c1542948af6161a5e8445cdf3a153d91968a57e330383f6a0df522d92f09dda453c8da0cb999ecee6c0c9998e2a2a702a98cef768e88b0383ddea9f7aa139a893b18381eb9166251ca35738750d67320fca43ad99d44a4f59317d878b8b98b469c08dc95728ed17d6b0cb42b412341f9afbe5955408097fcd32d12b0984157b723464843d62b2fe06102173d2d92222205a84ebfb346a31ae5471838aa439a06fbbb022fec5f8cd8c23bd234adb4296dee93940adfec0d15010361e8fb92ce5770ddccc88826b08c5c6fa5791449d3cb022a486d02a0ea8f186ea4ad6ab1b34878612820ae73218016ab767a8af4c0196e83b5,publicassistance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F10128%2F5438389%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20241013%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20241013T074228Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D62907fe09439148347e81bbb036b47cda88bf876b6c554219338348143bacaeb580eceb5144e4bbd388dd9f8883775e3a5a3461b58bc215c8e055ca1a708d1eb5acb907218fe0737aac6362537fd81ee50d6382be04d2e079901742a83e3dc3470c7f3c0f9382483df34dfa29c88bd6e2a097d56038f2cbac149e2fee0b518844217d92084238decc068125d681da7beaf5ba9a5388d9ba4130985bfe4f92a4c05f65fe1e30efeaec42763b60ded76b73801950cf3cc237f7bf364e65d3abb9e95b4c1abd80efbf4638ac21a6fefa216c07d1b21be92062d4677f59afd313617381025c4c58411c549c8dfa8cdb85e6f03edaf81414d7f217ed12730e6c6b9bf'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


**This notebook is an exercise in the [Pandas](https://www.kaggle.com/learn/pandas) course.  You can reference the tutorial at [this link](https://www.kaggle.com/residentmario/creating-reading-and-writing).**

---


# Introduction

The first step in most data analytics projects is reading the data file. In this exercise, you'll create Series and DataFrame objects, both by hand and by reading data files.

Run the code cell below to load libraries you will need (including code to check your answers).

In [ ]:
import pandas as pd
pd.set_option('display.max_rows', 5)
from learntools.core import binder; binder.bind(globals())
from learntools.pandas.creating_reading_and_writing import *
print("Setup complete.")

Setup complete.


# Exercises

## 1.

In the cell below, create a DataFrame `fruits` that looks like this:

![](https://storage.googleapis.com/kaggle-media/learn/images/Ax3pp2A.png)

In [ ]:
# Your code goes here. Create a dataframe matching the above diagram and assign it to the variable fruits.
fruits = pd.DataFrame({'Apples':[30], 'Bananas':[21]})

# Check your answer
q1.check()
fruits

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,Apples,Bananas
0,30,21


In [ ]:
#q1.hint()
#q1.solution()

## 2.

Create a dataframe `fruit_sales` that matches the diagram below:

![](https://storage.googleapis.com/kaggle-media/learn/images/CHPn7ZF.png)

In [ ]:
# Your code goes here. Create a dataframe matching the above diagram and assign it to the variable fruit_sales.
fruit_sales = pd.DataFrame({'Apples': [35, 41],
                           'Bananas': [21, 34]},
                          index = ['2017 Sales', '2018 Sales'])

# Check your answer
q2.check()
fruit_sales

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,Apples,Bananas
2017 Sales,35,21
2018 Sales,41,34


In [ ]:
#q2.hint()
#q2.solution()

## 3.

Create a variable `ingredients` with a Series that looks like:

```
Flour     4 cups
Milk       1 cup
Eggs     2 large
Spam       1 can
Name: Dinner, dtype: object
```

In [ ]:
ingredients = pd.Series(['4 cups', '1 cup', '2 large', '1 can'],
                       index = ['Flour', 'Milk', 'Eggs', 'Spam'],
                       name = "Dinner")

# Check your answer
q3.check()
ingredients

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

Flour     4 cups
Milk       1 cup
Eggs     2 large
Spam       1 can
Name: Dinner, dtype: object

In [ ]:
#q3.hint()
#q3.solution()

## 4.

Read the following csv dataset of wine reviews into a DataFrame called `reviews`:

![](https://storage.googleapis.com/kaggle-media/learn/images/74RCZtU.png)

The filepath to the csv file is `../input/wine-reviews/winemag-data_first150k.csv`. The first few lines look like:

```
,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,"This tremendous 100% varietal wine[...]",Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and[...]",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,,Tinta de Toro,Bodega Carmen Rodríguez
```

In [ ]:
reviews = pd.read_csv("../input/wine-reviews/winemag-data_first150k.csv", index_col =0 )
# Check your answer
q4.check()
reviews

<IPython.core.display.Javascript object>

<span style="color:#33cc33">Correct</span>

,country,description,designation,points,price,province,region_1,region_2,variety,winery
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,NaN,Tinta de Toro,Bodega Carmen Rodríguez
...,...,...,...,...,...,...,...,...,...,...
150928,France,"A perfect salmon shade, with scents of peaches...",Grand Brut Rosé,90,52.0,Champagne,Champagne,NaN,Champagne Blend,Gosset
150929,Italy,More Pinot Grigios should taste like this. A r...,NaN,90,15.0,Northeastern Italy,Alto Adige,NaN,Pinot Grigio,Alois Lageder


In [ ]:
#q4.hint()
#q4.solution()

## 5.

Run the cell below to create and display a DataFrame called `animals`:

In [ ]:
animals = pd.DataFrame({'Cows': [12, 20], 'Goats': [22, 19]}, index=['Year 1', 'Year 2'])
animals

,Cows,Goats
Year 1,12,22
Year 2,20,19


In the cell below, write code to save this DataFrame to disk as a csv file with the name `cows_and_goats.csv`.

In [ ]:
# Your code goes here
animals.to_csv(cows_and_goats.csv)
# Check your answer
q5.check()

In [ ]:
#q5.hint()
q5.solution()

<IPython.core.display.Javascript object>

<span style="color:#33cc99">Solution:</span> 
```python
animals.to_csv("cows_and_goats.csv")
```

# Keep going

Move on to learn about **[indexing, selecting and assigning](https://www.kaggle.com/residentmario/indexing-selecting-assigning)**.

---




*Have questions or comments? Visit the [course discussion forum](https://www.kaggle.com/learn/pandas/discussion) to chat with other learners.*